# Binance Trade Analysis
#### Analyze historical trade data from various Binance accounts over 90 days, containing:
Port_IDs: Unique identifiers for accounts.
Trade_History: Historical trades with details like timestamp, asset, side (BUY/SELL), price, and more.

Objective: Analyze the dataset to calculate financial metrics for each account, rank them, and provide a top 20 lis


## Import Dependencies 

In [4]:
import pandas as pd 
import numpy as np
import os
import json

In [5]:
file_path = "./TRADES_CopyTr_90D_ROI.csv"
trade_data = pd.read_csv(file_path)


# Data Exploration

In [6]:
trade_data.info()
trade_data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Port_IDs       150 non-null    int64 
 1   Trade_History  149 non-null    object
dtypes: int64(1), object(1)
memory usage: 2.5+ KB


,Port_IDs,Trade_History
0,3925368433214965504,"[{'time': 1718899656000, 'symbol': 'SOLUSDT', ..."
1,4002413037164645377,"[{'time': 1718980078000, 'symbol': 'NEARUSDT',..."
2,3923766029921022977,"[{'time': 1718677164000, 'symbol': 'ETHUSDT', ..."
3,3994879592543698688,"[{'time': 1718678214000, 'symbol': 'ETHUSDT', ..."
4,3926423286576838657,"[{'time': 1718979615000, 'symbol': 'ETHUSDT', ..."


# Handling Missing Values

In [7]:
# Data Parsing
import ast

# Parse the Trade_History of the first non-null entry to inspect the structure
sample_trade_history = trade_data['Trade_History'].dropna().iloc[0]
parsed_trade_history = ast.literal_eval(sample_trade_history)

# Display the first few trade details for inspection
# parsed_trade_history


In [8]:
# Display the raw string of the sample trade history to identify issues
print(sample_trade_history[:500])  # Show the first 500 characters for inspection


[{'time': 1718899656000, 'symbol': 'SOLUSDT', 'side': 'BUY', 'price': 132.537, 'fee': -0.9940275, 'feeAsset': 'USDT', 'quantity': 1988.055, 'quantityAsset': 'USDT', 'realizedProfit': 0.0, 'realizedProfitAsset': 'USDT', 'baseAsset': 'SOL', 'qty': 15.0, 'positionSide': 'LONG', 'activeBuy': True}, {'time': 1718899618000, 'symbol': 'DOGEUSDT', 'side': 'BUY', 'price': 0.12182, 'fee': -0.27979617, 'feeAsset': 'USDT', 'quantity': 1398.98088, 'quantityAsset': 'USDT', 'realizedProfit': 0.0, 'realizedProf


## Data Cleaning and Parsing

In [10]:
# Enhanced cleaning and parsing function for Trade_History
def clean_and_parse_trade_history_advanced(trade_history_str):
    try:
        # Replace single quotes with double quotes and ensure JSON compliance
        cleaned_str = trade_history_str.replace("'", '"').replace("True", "true").replace("False", "false").replace("None", "null")
        return json.loads(cleaned_str)
    except json.JSONDecodeError as e:
        return None  # Return None if parsing fails

# Apply the enhanced cleaning function to the sample trade history
parsed_trade_history = clean_and_parse_trade_history_advanced(sample_trade_history)

# Display the cleaned and parsed trade history

# parsed_trade_history[:5] if parsed_trade_history else "Parsing failed"



In [12]:
# Print the entire raw string of the sample for detailed inspection

# print(sample_trade_history)

In [13]:
# Parse all Trade_History entries in the dataset
trade_data['Parsed_Trade_History'] = trade_data['Trade_History'].apply(
    lambda x: clean_and_parse_trade_history_advanced(x) if pd.notnull(x) else None
)

# Check the success of parsing and count parsed entries
parsed_count = trade_data['Parsed_Trade_History'].notnull().sum()
total_entries = len(trade_data)

parsed_count, total_entries  # Display the number of successfully parsed entries


(149, 150)

## Calculate Metrics

In [14]:
import numpy as np

# Function to calculate metrics for a given trade history
def calculate_metrics(trade_history):
    if not trade_history:
        return None  # Skip if the trade history is missing

    # Initialize metrics
    realized_profits = []
    capital = []
    win_positions = 0
    total_positions = 0

    for trade in trade_history:
        realized_profits.append(trade['realizedProfit'])
        capital.append(abs(trade['quantity']))
        if trade['realizedProfit'] > 0:
            win_positions += 1
        total_positions += 1

    # Convert lists to numpy arrays for calculations
    realized_profits = np.array(realized_profits)
    capital = np.array(capital)

    # Calculate metrics
    roi = sum(realized_profits) / sum(capital) if sum(capital) != 0 else 0
    pnl = sum(realized_profits)
    win_rate = win_positions / total_positions if total_positions > 0 else 0

    return {
        'ROI': roi,
        'PnL': pnl,
        'Win Rate': win_rate,
        'Win Positions': win_positions,
        'Total Positions': total_positions
    }

# Apply metrics calculation to all accounts
trade_data['Metrics'] = trade_data['Parsed_Trade_History'].apply(calculate_metrics)

# Expand metrics into separate columns
metrics_df = trade_data.dropna(subset=['Metrics']).copy()
metrics_df = pd.concat(
    [metrics_df.drop(columns=['Metrics']), pd.json_normalize(metrics_df['Metrics'])],
    axis=1
)



## Calculate score

In [15]:
# Define a ranking score based on weighted metrics
def calculate_score(row):
    # Weights for metrics (can be adjusted based on priorities)
    weights = {
        'ROI': 0.4,
        'PnL': 0.3,
        'Win Rate': 0.2,
        'Win Positions': 0.1
    }
    return (
        row['ROI'] * weights['ROI'] +
        row['PnL'] * weights['PnL'] +
        row['Win Rate'] * weights['Win Rate'] +
        row['Win Positions'] * weights['Win Positions']
    )

# Add a score column to the dataframe
metrics_df['Score'] = metrics_df.apply(calculate_score, axis=1)

# Rank the accounts and extract the top 20
metrics_df['Rank'] = metrics_df['Score'].rank(ascending=False, method='dense')
top_20_accounts = metrics_df.sort_values(by='Score', ascending=False).head(20)

# Export the metrics and top 20 accounts to CSV
metrics_file_path = 'Result-Tables/Financial_Metrics.csv'
top_20_file_path = 'Result-Tables/Top_20_Accounts.csv'

metrics_df.to_csv(metrics_file_path, index=False)
top_20_accounts.to_csv(top_20_file_path, index=False)

metrics_file_path, top_20_file_path


('Result-Tables/Financial_Metrics.csv', 'Result-Tables/Top_20_Accounts.csv')

# Output 

In [16]:
metrics_df.head()

,Port_IDs,Trade_History,Parsed_Trade_History,ROI,PnL,Win Rate,Win Positions,Total Positions,Score,Rank
0,3.925368e+18,"[{'time': 1718899656000, 'symbol': 'SOLUSDT', ...","[{'time': 1718899656000, 'symbol': 'SOLUSDT', ...",0.003753,6789.436739,0.333333,486.0,1458.0,2085.499189,24.0
1,4.002413e+18,"[{'time': 1718980078000, 'symbol': 'NEARUSDT',...","[{'time': 1718980078000, 'symbol': 'NEARUSDT',...",0.024922,3686.969761,0.185764,107.0,576.0,1116.838050,53.0
2,3.923766e+18,"[{'time': 1718677164000, 'symbol': 'ETHUSDT', ...","[{'time': 1718677164000, 'symbol': 'ETHUSDT', ...",0.001565,772.729032,0.285147,359.0,1259.0,267.776365,133.0
3,3.994880e+18,"[{'time': 1718678214000, 'symbol': 'ETHUSDT', ...","[{'time': 1718678214000, 'symbol': 'ETHUSDT', ...",0.008283,3658.765062,0.552833,361.0,653.0,1133.843399,52.0
4,3.926423e+18,"[{'time': 1718979615000, 'symbol': 'ETHUSDT', ...","[{'time': 1718979615000, 'symbol': 'ETHUSDT', ...",0.001355,1309.459960,0.341004,163.0,478.0,409.206731,125.0


In [17]:
top_20_accounts.head()

,Port_IDs,Trade_History,Parsed_Trade_History,ROI,PnL,Win Rate,Win Positions,Total Positions,Score,Rank
82,4.020205e+18,"[{'time': 1718982983000, 'symbol': 'BTCUSDT', ...","[{'time': 1718982983000, 'symbol': 'BTCUSDT', ...",0.003116,71998.855953,0.329421,1993.0,6050.0,21799.023916,1.0
126,4.040383e+18,"[{'time': 1718848604000, 'symbol': 'MINAUSDT',...","[{'time': 1718848604000, 'symbol': 'MINAUSDT',...",0.005437,42574.473679,0.523220,2366.0,4522.0,13009.048922,2.0
23,4.021669e+18,"[{'time': 1718989823000, 'symbol': 'BTCUSDT', ...","[{'time': 1718989823000, 'symbol': 'BTCUSDT', ...",0.004044,26427.331592,0.313912,528.0,1682.0,7981.063878,3.0
64,3.960874e+18,"[{'time': 1718972086000, 'symbol': 'BTCUSDT', ...","[{'time': 1718972086000, 'symbol': 'BTCUSDT', ...",0.002333,19567.471286,0.515684,1233.0,2391.0,5993.645456,4.0
118,3.943534e+18,"[{'time': 1718899769000, 'symbol': 'RONINUSDT'...","[{'time': 1718899769000, 'symbol': 'RONINUSDT'...",0.010992,18015.997370,0.613971,2540.0,4137.0,5658.926402,5.0
